## 4 Assignment
a) 아래 예제 코드를 이용해 텍스트 입력의 숫자 변환 과정을 체크한다

b) testset의 임의 입력을 LongTensor로 변환해 학습 완료된 모델에 입력해보고, 결과가 어떠한지 체크한다 (하단의 출력 결과를 참조)



In [2]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data, datasets 
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
batch_size = 64
learning_rate = 0.001
training_epochs = 5

Mounted at /content/drive
cuda


In [3]:
# IMDB Dataset

# data.Field 설명 #
# sequential인자 : TEXT는 Sequential 데이터라 True, Lable은 비Sequential이라 False로 설정
# batch_first : Batch를 우선시 하여, Tensor 크기를 (BATCH_SIZE, 문장의 최대 길이)로 설정
# lower : 소문자 전환 인자

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

# data.Field.build_vocab() 라이브러리
# 문장 내 단어와 Integer index 를 매칭시키는 단어장(vocab)을 생성 == 워드 임베딩을 위한 Vocab 생성
# <UNK> = 0, <PAD> = 1 토큰도 추가.
# min_freq : 최소 5번 이상 등장한 단어들만 사전에 담겠다는 것. 
# 5번 미만으로 등장하는 단어는 UNK라는 토큰으로 대체

TEXT.build_vocab(trainset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(trainset)# LABEL 데이터를 기반으로 Vocab 생성

# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기
trainset, valset = trainset.split(split_ratio=0.8)
# 매 배치마다 비슷한 길이에 맞춰 줄 수 있도록 iterator 정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=batch_size,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 2 # Positive, Negative Class가 두 개

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:01<00:00, 47.6MB/s]


[TrainSet]: 20000 [ValSet]: 5000 [TestSet]: 25000 [Vocab]: 46159 [Classes] 2


In [4]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers # 일반적으로는 2

        #n_vocab : Vocab 안에 있는 단어의 개수, embed_dim : 임베딩 된 단어 텐서가 갖는 차원 값(dimension)
        self.embed = nn.Embedding(n_vocab, embed_dim)

        # hidden vector의 dimension과 dropout 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        #앞에서 정의한 하이퍼 파라미터를 넣어 GRU 정의
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        #Input: GRU의 hidden vector(context), Output : Class probability vector
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        # Input data: 한 batch 내 모든 영화 평가 데이터
        
        x = self.embed(x)# 영화 평 임베딩
        x, _ = self.gru(x)  # [i, b, h] 출력값 :  (batch_size, 입력 x의 길이, hidden_dim)

        # h_t : Batch 내 모든 sequential hidden state vector의 제일 마지막 토큰을 내포한 (batch_size, 1, hidden_dim)형태의 텐서 추출
        # 다른 의미로 영화 리뷰 배열들을 압축한 hidden state vector
        h_t = x[:,-1,:]

        self.dropout(h_t)# dropout 설정 후, 

        # linear layer의 입력으로 주고, 각 클래스 별 결과 logit을 생성.
        out = self.out(h_t)  # [b, h] -> [b, o]
        return out

In [4]:
# contruct model
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  avg_cost = 0
  for batch in train_iter:
    X, Y = batch.text.to(device), batch.label.to(device)
    Y.data.sub_(1)
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / batch_size
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

# model save
torch.save(model.state_dict(), '/content/drive/MyDrive/model_s1.pt')

[Epoch:    1] cost = 3.41286826
[Epoch:    2] cost = 3.39303303
[Epoch:    3] cost = 3.29124904
[Epoch:    4] cost = 2.28365111
[Epoch:    5] cost = 1.29460239
Learning Finished!


In [5]:
# model load
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_s1.pt'))

corrects = 0
for batch in val_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 

print('accuracy = ', corrects/len(val_iter.dataset)*100.0)

accuracy =  tensor(86.2200, device='cuda:0')


In [5]:
input_text, input_label = testset[3].text, testset[3].label
print(input_text)
print(TEXT.vocab[input_text[0]])
print(TEXT.vocab[input_text[1]])
print(TEXT.vocab[input_text[2]])
print(TEXT.vocab[input_text[3]])
print(TEXT.vocab[input_text[4]])

['the', 'word', 'impossible', 'has', 'led', 'many', 'to', 'select', 'a', 'particular', 'view', 'concerning', 'any', 'incredible', 'task.', 'in', '1927,', 'it', 'was', 'believed', 'no', 'man', 'could', 'fly', 'the', 'breath', 'of', 'the', 'atlantic', 'ocean.', 'many', 'had', 'tried', 'but', 'failed', 'and', 'some', 'even', 'gave', 'their', 'lives', 'to', 'the', 'effort.', 'nevertheless,', 'it', 'had', 'to', 'be', 'done', 'as', 'every', 'challenge', 'needs', 'to', 'be', 'met', 'with', 'equal', 'determination.', 'such', 'then', 'is', 'the', 'heart', 'of', 'this', 'movie', 'called', '"the', 'spirit', 'of', 'st.', 'louis."', 'the', 'actor', 'chosen', 'for', 'this', 'historic', 'film', 'is', 'none', 'other', 'than', "america's", 'own', 'james', 'stewart', 'who', 'convincingly', 'plays', 'charles', 'lindbergh.', 'although', 'there', 'are', 'many', 'facets', 'of', "lindbergh's", 'life,', 'the', 'segment', 'featured', 'here', 'is', 'his', 'efforts', 'to', 'be', 'the', 'first', 'man', 'to', 'fly

In [7]:
# load test text
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_s1.pt'))

input_embdedding = [TEXT.vocab[i] for i in input_text]

In [9]:
print(input_embdedding)
print(input_label)
input_embdedding = torch.tensor(input_embdedding, device=device, dtype=torch.long)
print(input_embdedding)

input_embdedding = input_embdedding.unsqueeze(0)
pred = model_new(input_embdedding)
print(pred.max(1)[1])

[2, 822, 1241, 41, 1563, 100, 6, 15279, 3, 928, 798, 3842, 92, 1172, 13361, 8, 0, 12, 14, 3082, 60, 160, 90, 2698, 2, 3450, 5, 2, 13867, 31349, 100, 62, 738, 18, 1272, 4, 45, 55, 441, 59, 517, 6, 2, 3966, 3853, 12, 62, 6, 28, 247, 15, 158, 3285, 677, 6, 28, 2036, 17, 3594, 42121, 130, 96, 7, 2, 657, 5, 10, 20, 437, 222, 1348, 5, 5472, 0, 2, 342, 2398, 16, 10, 6315, 24, 7, 603, 80, 68, 5377, 209, 569, 1729, 36, 5814, 270, 1489, 0, 281, 52, 22, 100, 17618, 5, 0, 747, 2, 2615, 2516, 198, 7, 21, 2864, 6, 28, 2, 82, 160, 6, 2698, 563, 2, 0, 2, 75, 7, 32, 246, 30, 4, 16, 0, 516, 1660, 6, 142, 2, 5127, 3160, 221, 13181, 6, 1713, 3, 301, 0, 6, 2, 12791, 2543, 6, 1769, 2, 1540, 19, 3, 8767, 352, 8, 186, 0, 0, 0, 15, 27, 14, 0, 13970, 4354, 0, 63, 22, 2516, 8, 10, 1099, 139, 80, 0, 63, 1609, 91, 10, 24, 1155, 22, 7178, 6401, 36, 270, 8153, 0, 0, 6894, 15, 1075, 43689, 1930, 1060, 4, 1489, 25320, 15, 0, 0, 2, 3184, 917, 5, 10, 204, 772, 20, 7, 11, 446, 350, 0, 19, 83, 9096, 12, 41, 227, 359, 3, 4